In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/4.src'):
    shutil.rmtree('./file/4.src')

os.makedirs('./file/4.src')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/3.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/3.part/{file}',
                           header=0,
                           dtype=str).fillna('')
    
    input_['No'] = input_['No'].astype(int)
    input_['rowId'] = input_['rowId'].astype(int)
    input_['qtyEach'] = input_['qtyEach'].astype(int)
    input_['maxRows'] = input_['maxRows'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =


    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                b = 0
                while True:
                    b += 1
                    
                    try:
                        resp = requests.get(f'''https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?partNum={input_.loc[a, 'Part Number']}&brand={input_.loc[a, 'brand']}&zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960''',
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))

                        if resp.status_code == 200:
                            break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                list_src = resp.json()
                list_src = [src['image_BG_Url'].strip() for src in list_src]

                # = = = = = = = = = = = = = = =

                dict_src = {}
                for i, src in zip(range(len(list_src)), list_src):
                    dict_src[str(i)] = src

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)

                df_temp['Json_Src'] = json.dumps(dict_src)
                
                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()

            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'No']}.{input_.loc[a, 'Part Number']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(5):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['No'],
                                                    ascending=[True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./file/4.src/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-src_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/4.src'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/4.src/{file}',
              f'''./file/4.src/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

[状态：ok，尝试次数：1] - 99.94% - crawler_998 > 1.HFG101
[剩余数量：0] - [当前时间：04:08:15]

输出ing...

总数量：19

[状态：ok，尝试次数：1] - 100.00% - crawler_999 > 5.HN17T
[剩余数量：14] - [当前时间：04:08:15]

[状态：ok，尝试次数：1] - 100.00% - crawler_999 > 4.HN-17
[剩余数量：13] - [当前时间：04:08:16]

[状态：ok，尝试次数：1] - 100.00% - crawler_999 > 3.HN16T
[剩余数量：12] - [当前时间：04:08:16]

[状态：ok，尝试次数：1] - 100.00% - crawler_999 > 6.HN-18
[剩余数量：11] - [当前时间：04:08:17]

[状态：ok，尝试次数：1] - 100.00% - crawler_999 > 7.HN-19
[剩余数量：10] - [当前时间：04:08:17]

[状态：ok，尝试次数：1] - 100.00% - crawler_999 > 2.HN-16
[剩余数量：9] - [当前时间：04:08:17]

[状态：ok，尝试次数：1] - 100.00% - crawler_999 > 8.HN19T
[剩余数量：8] - [当前时间：04:08:17]

[状态：ok，尝试次数：1] - 100.00% - crawler_999 > 9.HN-20
[剩余数量：7] - [当前时间：04:08:18]

[状态：ok，尝试次数：1] - 100.00% - crawler_999 > 1.HN-15
[剩余数量：6] - [当前时间：04:08:18]

[状态：ok，尝试次数：1] - 100.00% - crawler_999 > 10.HN30
[剩余数量：5] - [当前时间：04:08:18]

[状态：ok，尝试次数：1] - 100.00% - crawler_999 > 11.HN35
[剩余数量：4] - [当前时间：04:08:19]

[状态：ok，尝试次数：1] - 100.00% - crawler_999 > 13.HN38
[剩余数

Progress: 100%|████████████████████████| 1574/1574 [00:00<00:00, 6894.41it/s]

Done ~
